In [3]:
!pip install librosa
!pip install sounddevice soundfile numpy

In [2]:
import sounddevice as sd
import numpy as np
import queue
import time

SAMPLE_RATE = 16000 # Please don't change
WINDOW_SIZE = 3.0
STRIDE = 1.0

MIC_RATE = 44100

WINDOW_SAMPLES = SAMPLE_RATE * WINDOW_SIZE
STRIDE_SAMPLES = SAMPLE_RATE * STRIDE


<h3>MEL</h3>

In [1]:
import librosa
def generate_mel(y):
    mel = librosa.feature.melspectrogram(
            y=y,
            sr=SAMPLE_RATE,
            n_fft=512,
            hop_length=160,
            n_mels=40
        )
    mel = librosa.power_to_db(mel, ref=np.max).astype(np.float32)
    return torch.from_numpy(mel).float()

<h3>Model</h3>

In [4]:
import torch
from model.crnn_with_mbconv import CRNN_with_MBConv

        
model = CRNN_with_MBConv()
model.load_state_dict(torch.load("trained_models/worst_crnn_with_mbconv_model.pth"))
model.eval()

def wake_word_detect(data):
    mel = generate_mel(data).unsqueeze(0).unsqueeze(0)
    with torch.no_grad():
        out = model(mel).squeeze()
        print(out)
        pred = (out > 0.5).float()
        if pred == 1.0:
            return True
        else:
            return False

C:\Users\danma\AppData\Local\Temp\ipykernel_39336\294880652.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("trained_models/worst_crnn_w

<h2>Audio Recording</h2>
Starts audio recording and detect wake word
<h3>Say: "Hey, Snips!"</h3>

In [5]:
audio_queue = queue.Queue()
audio_buffer = []

tried_to_detect = False
def callback(indata, frames, time, status):
    if status:
        print(status)
    audio_queue.put(indata.copy().flatten())

with sd.InputStream(samplerate=SAMPLE_RATE,callback=callback):
    try:
        while True:    
            new_audio = audio_queue.get()
            #new_audio = librosa.resample(new_audio, orig_sr=MIC_RATE, target_sr=SAMPLE_RATE)
            if new_audio is None:
                continue
            audio_buffer = np.concatenate((audio_buffer, new_audio))
            if len(audio_buffer) > WINDOW_SAMPLES and not tried_to_detect:
                is_detected = wake_word_detect(audio_buffer)
                if (is_detected) :
                    print("Wake Word Detected!")
                    sd.play(audio_buffer, SAMPLE_RATE)
                    break
                tried_to_detect = True
            if len(audio_buffer) >= WINDOW_SAMPLES + STRIDE_SAMPLES:
                audio_buffer = audio_buffer[int(STRIDE_SAMPLES):]
                tried_to_detect = False
                
    except KeyboardInterrupt:
        print("Exiting from keyboard interrupt")
        

tensor(-0.0380)
tensor(0.0084)
tensor(-0.1541)
tensor(1.0057)
Wake Word Detected!
